In [4]:
%%writefile parallel_bfs_dfs.cpp
#include <iostream>
#include <vector>
#include <queue>
#include <stack>
#include <omp.h>
using namespace std;

// Parallel BFS using OpenMP
void parallelBFS(vector<vector<int>>& graph, int start) {
    int n = graph.size();
    vector<bool> visited(n, false);
    queue<int> q;

    visited[start] = true;
    q.push(start);

    cout << "Parallel BFS traversal: ";

    while (!q.empty()) {
        int size = q.size();

        #pragma omp parallel for
        for (int i = 0; i < size; ++i) {
            int current;

            #pragma omp critical
            {
                if (!q.empty()) {
                    current = q.front();
                    q.pop();
                    cout << current << " ";
                }
            }

            // Parallelizing the exploration of neighbors
            #pragma omp parallel for
            for (int j = 0; j < graph[current].size(); j++) {
                int neighbor = graph[current][j];
                #pragma omp critical
                {
                    if (!visited[neighbor]) {
                        visited[neighbor] = true;
                        q.push(neighbor);
                    }
                }
            }
        }
    }
    cout << endl;
}

// Parallel DFS using OpenMP
void parallelDFS(vector<vector<int>>& graph, int start) {
    int n = graph.size();
    vector<bool> visited(n, false);
    stack<int> s;
    s.push(start);

    cout << "Parallel DFS traversal: ";

    while (!s.empty()) {
        int current;
        bool valid = false;

        #pragma omp critical
        {
            if (!s.empty()) {
                current = s.top();
                s.pop();
                valid = true;
            }
        }

        if (!valid) continue;

        if (!visited[current]) {
            visited[current] = true;
            cout << current << " ";

            // Parallelizing exploration of neighbors
            #pragma omp parallel for
            for (int i = 0; i < graph[current].size(); i++) {
                int neighbor = graph[current][i];
                #pragma omp critical
                {
                    if (!visited[neighbor]) {
                        s.push(neighbor);
                    }
                }
            }
        }
    }
    cout << endl;
}

int main() {
    vector<vector<int>> graph = {
        {1, 2},
        {0, 3, 4},
        {0, 4},
        {1, 5},
        {1, 2, 5},
        {3, 4}
    };

    int startNode = 0;

    cout << "Graph traversal using OpenMP from node " << startNode << ":\n";
    parallelBFS(graph, startNode);
    parallelDFS(graph, startNode);

    return 0;
}


Overwriting parallel_bfs_dfs.cpp


In [3]:
!g++ -fopenmp parallel_bfs_dfs.cpp -o parallel_bfs_dfs
!./parallel_bfs_dfs


Graph traversal using OpenMP from node 0:
Parallel BFS traversal: 0 1 2 4 3 5 
Parallel DFS traversal: 0 1 3 5 4 2 
